# Import

In [1]:
import torch
import torch.nn as nn
from multiprocessing import Pool

import math, random, sys
import cPickle as pickle
import argparse

from fast_jtnn import *
import rdkit

# Write functions

In [2]:
def tensorize(smiles, assm=False):
    try: #added
        mol_tree = MolTree(smiles)
        mol_tree.recover()
        if assm:
            mol_tree.assemble()
            for node in mol_tree.nodes:
                if node.label not in node.cands:
                    node.cands.append(node.label)

        del mol_tree.mol
        for node in mol_tree.nodes:
            del node.mol
            del node.clique

        return mol_tree
    except: #added
        return None

In [3]:
def tensorize_pair(smiles_pair):
    mol_tree0 = tensorize(smiles_pair[0], assm=True) #added
    #mol_tree0 = tensorize(smiles_pair[0], assm=False)
    #mol_tree1 = tensorize(smiles_pair[1], assm=False) #added
    mol_tree1 = tensorize(smiles_pair[1], assm=True)
    return (mol_tree0, mol_tree1)

# Manually set args

In [4]:
ncpu = 12

In [9]:
train = '/home/rgur/CS6250_project/g2g/polymers_trial2/data/trial_train_1337.txt'

# Manually run main for pair

In [10]:
lg = rdkit.RDLogger.logger() 
lg.setLevel(rdkit.RDLogger.CRITICAL)

#parser = argparse.ArgumentParser()
#parser.add_argument('--train', required=True)
#parser.add_argument('--mode', type=str, default='pair')
#parser.add_argument('--ncpu', type=int, default=8)
#args = parser.parse_args()

#ncpu = args.ncpu

In [11]:
pool = Pool(ncpu)

In [12]:
#train = args.train
#if args.mode == 'pair':
    #dataset contains molecule pairs
with open(train) as f:
    data = [line.strip("\r\n ").split()[:2] for line in f]

In [13]:
len(data)

1337

In [14]:
#all_data = pool.map(tensorize_pair, data)
all_data1 = pool.map(tensorize_pair, data) #added

In [15]:
len(all_data1)

1337

In [16]:
all_data = [i for i in all_data1 if None not in i] #added

In [17]:
len(all_data)

1226

In [18]:
num_splits = len(all_data) / 2000 #added
num_splits

0

In [19]:
if num_splits == 0: #added
    num_splits = 1 #added

In [20]:
le = (len(all_data) + num_splits - 1) / num_splits
le

1226

In [21]:
for split_id in xrange(num_splits):
    st = split_id * le
    sub_data = all_data[st : st + le]

    with open('tensors-%d.pkl' % split_id, 'wb') as f:
        pickle.dump(sub_data, f, pickle.HIGHEST_PROTOCOL)

# elif args.mode == 'single':
#     #dataset contains single molecules
#     with open(args.train) as f:
#         data = [line.strip("\r\n ").split()[0] for line in f]

#     all_data = pool.map(tensorize, data)
#     num_splits = len(data) / 10000

#     le = (len(all_data) + num_splits - 1) / num_splits

#     for split_id in xrange(num_splits):
#         st = split_id * le
#         sub_data = all_data[st : st + le]

#         with open('tensors-%d.pkl' % split_id, 'wb') as f:
#             pickle.dump(sub_data, f, pickle.HIGHEST_PROTOCOL)

# Manually run main for single

Manually set args

In [22]:
train = '/home/rgur/CS6250_project/g2g/polymers_trial2/data/trial_test_899.txt'

In [23]:
#args.train = train #added

In [24]:
with open(train) as f:
    data = [line.strip("\r\n ").split()[0] for line in f]

In [25]:
#all_data = pool.map(tensorize, data)
all_data1 = pool.map(tensorize, data) #added

In [26]:
len(all_data1)

899

In [27]:
all_data = [i for i in all_data1 if i is not None] #added

In [28]:
len(all_data)

607

In [29]:
num_splits = len(all_data) / 10000 #added

In [30]:
if num_splits == 0: #added
    num_splits = 1 #added

In [31]:
le = (len(all_data) + num_splits - 1) / num_splits

In [32]:
le

607

In [33]:
for split_id in xrange(num_splits):
    st = split_id * le
    sub_data = all_data[st : st + le]

    with open('tensors-%d.pkl' % split_id, 'wb') as f:
        pickle.dump(sub_data, f, pickle.HIGHEST_PROTOCOL)